In [2]:
from pyspark import SparkContext
from pyspark.ml.feature import OneHotEncoder, StringIndexer

from pyspark.sql import SQLContext
import matplotlib.pyplot as plt

sqlContext = SQLContext(sc)
data = sc.textFile('train.txt', 10000)
data = data.map(lambda x: x.split('\t'))

train10M, train38M = data.randomSplit([10, 38])
train5M, validation2M, test3M = train10M.randomSplit([5,2,3])

label = train5M.map(lambda x: x[0])
entire_train = train5M.map(lambda x: x[1:])

In [3]:
df_train = sqlContext.createDataFrame(train5M, ['C'+str(i) for i in range(39)])

In [4]:
def changeDataType(df):
    new = df    
    for i in range(13):
        new = new.withColumn('C'+str(i), new['C'+str(i)].cast('int'))
    return new

In [5]:
new_df = changeDataType(df_train)

In [6]:
new_df

DataFrame[C0: int, C1: int, C2: int, C3: int, C4: int, C5: int, C6: int, C7: int, C8: int, C9: int, C10: int, C11: int, C12: int, C13: string, C14: string, C15: string, C16: string, C17: string, C18: string, C19: string, C20: string, C21: string, C22: string, C23: string, C24: string, C25: string, C26: string, C27: string, C28: string, C29: string, C30: string, C31: string, C32: string, C33: string, C34: string, C35: string, C36: string, C37: string, C38: string, _40: string]

In [21]:
stringIndexer = StringIndexer(inputCol="C13", outputCol="C13Index")

In [22]:
cols = [new_df.select('C' + str(i)).rdd.flatMap(lambda x:x) for i in range(13, 39)]

In [24]:
cols[14]

PythonRDD[200] at RDD at PythonRDD.scala:43

In [49]:
col14 = sqlContext.createDataFrame(train5M.map(lambda x: [x[0],x[14]]), ['C'+str(0), 'C'+str(14)])

In [50]:
col14

DataFrame[C0: string, C14: string]

In [52]:
col14_filled = col14.na.replace('', 'unknown')

In [58]:
stringIndexer = StringIndexer(inputCol="C14", outputCol="categoryIndex")

In [59]:
model = stringIndexer.fit(col14)

In [60]:
indexed = model.transform(col14)

In [61]:
encoder = OneHotEncoder(dropLast=False, inputCol="categoryIndex", outputCol="categoryVec")

In [62]:
encoded = encoder.transform(indexed)

In [63]:
encoded.select("C0", "categoryVec").show()

+---+-----------------+
| C0|      categoryVec|
+---+-----------------+
|  0|(1451,[23],[1.0])|
|  0| (1451,[0],[1.0])|
|  0| (1451,[0],[1.0])|
|  0| (1451,[2],[1.0])|
|  1| (1451,[3],[1.0])|
|  0| (1451,[0],[1.0])|
|  0| (1451,[0],[1.0])|
|  0| (1451,[0],[1.0])|
|  0|(1451,[30],[1.0])|
|  0| (1451,[1],[1.0])|
|  0|(1451,[26],[1.0])|
|  0| (1451,[2],[1.0])|
|  1| (1451,[2],[1.0])|
|  0| (1451,[6],[1.0])|
|  0| (1451,[2],[1.0])|
|  1| (1451,[1],[1.0])|
|  0| (1451,[0],[1.0])|
|  0| (1451,[1],[1.0])|
|  1| (1451,[0],[1.0])|
|  0| (1451,[5],[1.0])|
+---+-----------------+
only showing top 20 rows



In [7]:
columns = [i for i in range(13, 39)]
cols = [new_df.select('C' + str(i)).rdd.flatMap(lambda x:x) for i in range(13, 39)]


In [ ]:
new_df = changeDataType(df_train)
columns = [i for i in range(13, 39)]
cols = [new_df.select('C' + str(i)).rdd.flatMap(lambda x:x) for i in range(13, 39)]
for col in columns:
    uniques = cols[col].distinct()
    keys = uniques.collect()
    bins = len(keys)
    dic = dict(zip(keys, range(bins)))
    convert = cols[col].map(lambda x:dic[x])
    hist = convert.histogram(bins)
    plt.bar(hist[0][1:], hist[1])
    # hist = new.select(col).rdd.flatMap(lambda x: x).histogram(20) 
    # plt.hist(hist[1], bins=hist[0])
    # plt.title(col)
    plt.savefig(str(col)+'.jpg')